In [33]:
# -*- coding: utf-8 -*-
#leemos las librerias para poder aplicar distintas funciones
library(readxl) 
library(dplyr)
library(stringr)
library(writexl)

In [34]:
#guardamos los archivos
Victoria <- read.csv("VICTORIA.csv")
primaria_2023 <- read.csv("Primaria 2023 - matrícula primaria.xls.csv")
secundaria_2023 <- read.csv("secundaria 2023 - xls_export.csv")
primaria_depta <- read.csv("primaria 2023 por departamentos - xls_export (10).csv")
secundaria_depta <- read.csv("secundaria por departamentos 2023 - xls_export (11).csv")
dim(Victoria) #dimension del archivo

[1] 320330     26

In [35]:
names(Victoria) #muestra las variables
#cambiamos los nombres de las variables para un mejor entendimiento
Victoria <- rename(Victoria, c("CodigoUnicoEscolar" = "CUE", "ModEnseñanza" = "ModEnsenaza", "año" = "anioLectivo", "esMultiaño" ="esMultianio", "IdModEnseñanza" = "IdModEnsenanza","NivelEnseñanza" = "NivelEnsenanza"))
str(Victoria) #muestra los nombres del dataset, nos ayuda a identificar el tipo de variables

[1] "X"                  "CUE"                "departamento"      
 [4] "localidad"          "CodigoPostal"       "EsPrivada"         
 [7] "turno"              "curso"              "periodoEvaluatorio"
[10] "idalumno"           "Documento"          "anioLectivo"       
[13] "asignatura"         "nota"               "observaciones"     
[16] "idSubOrganizacion"  "idDivision"         "NivelEnsenanza"    
[19] "orden"              "idOrganizaciones"   "ddivision"         
[22] "Nivel"              "esMultianio"        "IdModEnsenanza"    
[25] "Modalidad"          "ModEnsenaza"

'data.frame':	320330 obs. of  26 variables:
 $ X                 : int  1 2 3 4 5 6 7 8 9 10 ...
 $ CodigoUnicoEscolar: int  9002121 9002121 9002121 9002121 9002121 9002121 9002121 9002121 9002121 9002121 ...
 $ departamento      : chr  "VICTORIA" "VICTORIA" "VICTORIA" "VICTORIA" ...
 $ localidad         : chr  "VICTORIA" "VICTORIA" "VICTORIA" "VICTORIA" ...
 $ CodigoPostal      : int  3151 3151 3151 3151 3151 3151 3151 3151 3151 3151 ...
 $ EsPrivada         : chr  "N" "N" "N" "N" ...
 $ turno             : chr  "TARDE" "TARDE" "TARDE" "TARDE" ...
 $ curso             : chr  "4 - M Prim" "4 - M Prim" "4 - M Prim" "4 - M Prim" ...
 $ periodoEvaluatorio: chr  "Primer Trimestre" "Primer Trimestre" "Primer Trimestre" "Primer Trimestre" ...
 $ idalumno          : int  672655 672655 672655 672655 852106 852106 852106 852106 852106 852106 ...
 $ Documento         : int  53400853 53400853 53400853 53400853 53799627 53799627 53799627 53799627 53799627 53799627 ...
 $ año               : int  2

In [36]:
#descubrimos que al intentar modificar el nombre de una varible, esta se encuentra dos veces en el conjunto de datos
Victoria$ddivision <- NULL #entonces procedemos a eliminar una columna
names(Victoria) #verificamos si se elimino correctamente

[1] "X"                  "CodigoUnicoEscolar" "departamento"      
 [4] "localidad"          "CodigoPostal"       "EsPrivada"         
 [7] "turno"              "curso"              "periodoEvaluatorio"
[10] "idalumno"           "Documento"          "año"               
[13] "asignatura"         "nota"               "observaciones"     
[16] "idSubOrganizacion"  "idDivision"         "NivelEnseñanza"    
[19] "orden"              "idOrganizaciones"   "Nivel"             
[22] "esMultiaño"         "IdModEnseñanza"     "Modalidad"         
[25] "ModEnseñanza"

In [37]:
#reemplazamos las notas "EPAC", "UP" y "PA" con un valor menor a 6
Victoria$nota[Victoria$nota %in% c("EPAC")] <- 5
Victoria$nota[Victoria$nota %in% c("PA")] <- 6
Victoria$nota[Victoria$nota %in% c("UP")] <- 7
Victoria$nota <- suppressWarnings(as.numeric(Victoria$nota)) #convertimos la variable nota a tipo numérico

In [38]:
#ahora podemos dividir el dataset en dos
Victoria_primaria <- filter(Victoria, NivelEnseñanza == "PRIMARIO" & ( ModEnseñanza == "Comun"| ModEnseñanza == "Escuelas NINA") & Modalidad == "Primaria" & esMultiaño == "N") #se guardan los alumnos que cursan la primaria
Victoria_secundaria <- filter(Victoria, NivelEnseñanza == "SECUNDARIO" & (ModEnseñanza == "Comun" | ModEnseñanza == "Técnica" | ModEnseñanza =="Agrotécnica") & esMultiaño == "N" & Modalidad == "Secundaria") #se guardan los alumnos que cursan la secundaria
length(unique(Victoria_primaria$CodigoUnicoEscolar)) #cantidad de escuelas primarias en Victoria
length(unique(primaria_2023$CUE)) #total de escuelas primarias
length(unique(Victoria_secundaria$CodigoUnicoEscolar))#cantidad de escuelas secundarias en Victoria
length(unique(secundaria_2023$CUE))#total de escuelas secundarias

[1] 24

[1] 1261

[1] 21

[1] 578

In [39]:
dim(Victoria_primaria)
dim(Victoria_secundaria)

[1] 98613    25

[1] 128743     25

In [40]:
PrimerT_primaria <- filter(Victoria_primaria, periodoEvaluatorio == "Primer Trimestre")
PrimerT_primaria <- rename(PrimerT_primaria,NotasPrimerT = nota)
SegundoT_primaria <- filter(Victoria_primaria, periodoEvaluatorio == "Segundo Trimestre")
SegundoT_primaria <- rename(SegundoT_primaria,NotasSegundoT = nota)
TercerT_primaria <- filter(Victoria_primaria, periodoEvaluatorio == "Tercer Trimestre")
TercerT_primaria <- rename(TercerT_primaria,NotasTercerT = nota)

In [41]:
aux <- merge(x = PrimerT_primaria, y = SegundoT_primaria, by = c("Documento", "asignatura"))
auxiliar <- merge(x = aux, y = TercerT_primaria, by = c("Documento", "asignatura"))

In [42]:
Notas_Victoria_primaria <- select(auxiliar, 'CodigoUnicoEscolar', 'departamento', 'localidad', 'EsPrivada', 'turno' , 'curso' , 'periodoEvaluatorio' , 'Documento', 'asignatura','idalumno', 'observaciones', 'idSubOrganizacion', 'idDivision', 'NivelEnseñanza', 'orden','Nivel', 'esMultiaño', 'IdModEnseñanza', 'Modalidad', 'ModEnseñanza', 'año','NotasPrimerT','NotasSegundoT', 'NotasTercerT')

In [43]:
Notas_Victoria_primaria$Promedio <-round ((as.numeric(Notas_Victoria_primaria$NotasPrimerT)+as.numeric(Notas_Victoria_primaria$NotasSegundoT)+as.numeric(Notas_Victoria_primaria$NotasTercerT))/3,2)
head(Notas_Victoria_primaria)

,CodigoUnicoEscolar,departamento,localidad,EsPrivada,turno,curso,periodoEvaluatorio,Documento,asignatura,idalumno,...,Nivel,esMultiaño,IdModEnseñanza,Modalidad,ModEnseñanza,año,NotasPrimerT,NotasSegundoT,NotasTercerT,Promedio
,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<int>,...,<int>,<chr>,<int>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
1,9000753,VICTORIA,VICTORIA,N,MAÑANA,6 - B Prim,Tercer Trimestre,49438652,ARTES VISUALES,512667,...,2,N,5,Primaria,Comun,2023,9,5,8,7.33
2,9000753,VICTORIA,VICTORIA,N,MAÑANA,6 - B Prim,Tercer Trimestre,49438652,CIENCIAS NATURALES,512667,...,2,N,5,Primaria,Comun,2023,6,6,6,6.00
3,9000753,VICTORIA,VICTORIA,N,MAÑANA,6 - B Prim,Tercer Trimestre,49438652,CIENCIAS SOCIALES,512667,...,2,N,5,Primaria,Comun,2023,6,6,6,6.00
4,9000753,VICTORIA,VICTORIA,N,MAÑANA,6 - B Prim,Tercer Trimestre,49438652,EDUCACION FISICA,512667,...,2,N,5,Primaria,Comun,2023,7,8,7,7.33
5,9000753,VICTORIA,VICTORIA,N,MAÑANA,6 - B Prim,Tercer Trimestre,49438652,EDUCACION MUSICAL,512667,...,2,N,5,Primaria,Comun,2023,8,7,7,7.33
6,9000753,VICTORIA,VICTORIA,N,MAÑANA,6 - B Prim,Tercer Trimestre,49438652,EDUCACION TECNOLOGICA,512667,...,2,N,5,Primaria,Comun,2023,7,6,7,6.67


In [44]:
Notas_Victoria_primaria <- Notas_Victoria_primaria %>% mutate(Condición = case_when(
  Promedio >= 6 ~ "Aprobado",
  Promedio <=6  ~ "Rinde",
   TRUE~ "sin registro completo"))
head(Notas_Victoria_primaria)

,CodigoUnicoEscolar,departamento,localidad,EsPrivada,turno,curso,periodoEvaluatorio,Documento,asignatura,idalumno,...,esMultiaño,IdModEnseñanza,Modalidad,ModEnseñanza,año,NotasPrimerT,NotasSegundoT,NotasTercerT,Promedio,Condición
,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<int>,...,<chr>,<int>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,9000753,VICTORIA,VICTORIA,N,MAÑANA,6 - B Prim,Tercer Trimestre,49438652,ARTES VISUALES,512667,...,N,5,Primaria,Comun,2023,9,5,8,7.33,Aprobado
2,9000753,VICTORIA,VICTORIA,N,MAÑANA,6 - B Prim,Tercer Trimestre,49438652,CIENCIAS NATURALES,512667,...,N,5,Primaria,Comun,2023,6,6,6,6.00,Aprobado
3,9000753,VICTORIA,VICTORIA,N,MAÑANA,6 - B Prim,Tercer Trimestre,49438652,CIENCIAS SOCIALES,512667,...,N,5,Primaria,Comun,2023,6,6,6,6.00,Aprobado
4,9000753,VICTORIA,VICTORIA,N,MAÑANA,6 - B Prim,Tercer Trimestre,49438652,EDUCACION FISICA,512667,...,N,5,Primaria,Comun,2023,7,8,7,7.33,Aprobado
5,9000753,VICTORIA,VICTORIA,N,MAÑANA,6 - B Prim,Tercer Trimestre,49438652,EDUCACION MUSICAL,512667,...,N,5,Primaria,Comun,2023,8,7,7,7.33,Aprobado
6,9000753,VICTORIA,VICTORIA,N,MAÑANA,6 - B Prim,Tercer Trimestre,49438652,EDUCACION TECNOLOGICA,512667,...,N,5,Primaria,Comun,2023,7,6,7,6.67,Aprobado


In [45]:
table(Notas_Victoria_primaria$asignatura)
head(Notas_Victoria_primaria$asignatura,10)
Notas_Victoria_primaria$asignatura <- toupper(Notas_Victoria_primaria$asignatura)
head(Notas_Victoria_primaria$asignatura,10)


                      Ac.en Lengua y Cs. Sociales 
                                               54 
                 Ac.en Matemática y Cs. Naturales 
                                               54 
                        Acompañamiento al Estudio 
                                               29 
                 Acompañamiento al Estudio Lengua 
                                               66 
             Acompañamiento al Estudio Matemática 
                                               66 
              Acompañamiento al Estudio Naturales 
                                               66 
               Acompañamiento al Estudio Sociales 
                                               66 
                                   ARTES VISUALES 
                                             3264 
                                       Catequesis 
                                              202 
                                       CATEQUESIS 
                              

[1] "ARTES VISUALES"        "CIENCIAS NATURALES"    "CIENCIAS SOCIALES"    
 [4] "EDUCACION FISICA"      "EDUCACION MUSICAL"     "EDUCACION TECNOLOGICA"
 [7] "LENGUA"                "MATEMATICA"            "ARTES VISUALES"       
[10] "CIENCIAS NATURALES"

[1] "ARTES VISUALES"        "CIENCIAS NATURALES"    "CIENCIAS SOCIALES"    
 [4] "EDUCACION FISICA"      "EDUCACION MUSICAL"     "EDUCACION TECNOLOGICA"
 [7] "LENGUA"                "MATEMATICA"            "ARTES VISUALES"       
[10] "CIENCIAS NATURALES"

In [46]:
Notas_Victoria_primaria$asignatura <- gsub("MATEMÁTICA|ACOMPAÑAMIENTO AL ESTUDIO MATEMÁTICA", "MATEMATICA", Notas_Victoria_primaria$asignatura)
Notas_Victoria_primaria$asignatura <- gsub("LITERATURA|ACOMPAÑAMIENTO AL ESTUDIO LENGUA", "LENGUA", Notas_Victoria_primaria$asignatura)
Notas_Victoria_primaria$asignatura <- gsub("ACOMPAÑAMIENTO AL ESTUDIO SOCIALES", "CIENCIAS SOCIALES", Notas_Victoria_primaria$asignatura)
Notas_Victoria_primaria$asignatura <- gsub("ACOMPAÑAMIENTO AL ESTUDIO NATURALES|TALLER VIDA EN LA NATURALEZA", "CIENCIAS NATURALES", Notas_Victoria_primaria$asignatura)
Notas_Victoria_primaria$asignatura <- gsub("TALLER DE ARTES VISUALES|TALLER DE LENGUAJE DE LAS ARTES VISUALES", "ARTES VISUALES", Notas_Victoria_primaria$asignatura)
Notas_Victoria_primaria$asignatura <- gsub("TALLER DE DEPORTE|DEPORTE", "EDUCACION FISICA", Notas_Victoria_primaria$asignatura)
Notas_Victoria_primaria$asignatura <- gsub("E.D.I.", "EDI", Notas_Victoria_primaria$asignatura)
Notas_Victoria_primaria$asignatura <- gsub("TALLER DE MUSICA|TALLER DE FOLKLORE", "EDUCACION MUSICAL", Notas_Victoria_primaria$asignatura)
Notas_Victoria_primaria$asignatura <- gsub("TALLER DE TIC|TALLER DE TIC Y RECREACIÓN DE LA IMAGEN|TALLER TIC|TICS", "TIC", Notas_Victoria_primaria$asignatura)
Notas_Victoria_primaria$asignatura <- gsub("IDIOMA EXTRANJERO|INGLÉS|TALLER DE INGLES|TALLER DE INGLÉS|TALLER DE LENGUA EXTRANJERA|INGLEs|TALLER INGLES|TALLER LENGUA EXTRANJERA (INGLES)", "INGLES", Notas_Victoria_primaria$asignatura)
Notas_Victoria_primaria$asignatura <- gsub("TALLER DE TEATRO|TALLER TEATRO", "TALLER  DE TEATRO", Notas_Victoria_primaria$asignatura)
Notas_Victoria_primaria$asignatura <- gsub("INFORMÁTICA", "INFORMATICA", Notas_Victoria_primaria$asignatura)
Notas_Victoria_primaria$asignatura <- gsub("TALLER LABORATORIO", "TALLER DE LABORATORIO", Notas_Victoria_primaria$asignatura)
Notas_Victoria_primaria$asignatura <- gsub("AC.EN LENGUA Y CS. SOCIALES|T/A ESTUDIO (LENGUA-CS SOCIALES)|TALLER ACOMPAÑAMIENTO LENGUA Y CIENCIAS SOCIALES", "TALLER DE CS. SOCIALES Y LENGUA", Notas_Victoria_primaria$asignatura)
Notas_Victoria_primaria$asignatura <- gsub("AC.EN MATEMÁTICA Y CS. NATURALES|T/A ESTUDIO (MATEMATICA-CS NATURALES)|TALLER ACOMPAÑAMIENTO  MATEMÁTICA Y CS. NATURALES", "TALLER DE CS. NATURALES Y MATEMATICA", Notas_Victoria_primaria$asignatura)

In [47]:
table(Notas_Victoria_primaria$curso)
Notas_Victoria_primaria <- Notas_Victoria_primaria[Notas_Victoria_primaria$curso != "1-6 - A Prim", ]
table(Notas_Victoria_primaria$curso)


1-6 - A Prim   1 - A Prim   1 - B Prim   1 - C Prim   1 - D Prim   1 - E Prim 
           8         2502         1279          729          519          104 
  2 - A Prim   2 - B Prim   2 - C Prim   2 - D Prim   2 - U Prim  2° - A Prim 
        2260         1401          697          648          153           42 
  3 - A Prim   3 - B Prim   3 - C Prim   3 - D Prim   4 - A Prim   4 - B Prim 
        2622         1671          537          723         2157         1004 
  4 - C Prim   4 - D Prim   5 - A Prim   5 - B Prim   5 - C Prim   5 - D Prim 
         539          716         2364         1246          861          565 
  6 - A Prim   6 - B Prim  6 - C  Prim   6 - C Prim   6 - D Prim  6° - D Prim 
        2738         1475          108          530          608          153 


 1 - A Prim  1 - B Prim  1 - C Prim  1 - D Prim  1 - E Prim  2 - A Prim 
       2502        1279         729         519         104        2260 
 2 - B Prim  2 - C Prim  2 - D Prim  2 - U Prim 2° - A Prim  3 - A Prim 
       1401         697         648         153          42        2622 
 3 - B Prim  3 - C Prim  3 - D Prim  4 - A Prim  4 - B Prim  4 - C Prim 
       1671         537         723        2157        1004         539 
 4 - D Prim  5 - A Prim  5 - B Prim  5 - C Prim  5 - D Prim  6 - A Prim 
        716        2364        1246         861         565        2738 
 6 - B Prim 6 - C  Prim  6 - C Prim  6 - D Prim 6° - D Prim 
       1475         108         530         608         153 

In [48]:
Notas_Victoria_primaria <- Notas_Victoria_primaria[Notas_Victoria_primaria$Condición != "sin registro completo", ]

In [49]:
Notas_Victoria_primaria$curso <- as.character(Notas_Victoria_primaria$curso)
Notas_Victoria_primaria$Año_curso <- str_sub(Notas_Victoria_primaria$curso,1, 1)
print(head(Notas_Victoria_primaria$Año_curso),2)
table(Notas_Victoria_primaria$Año_curso)

[1] "6" "6" "6" "6" "6" "6"



   1    2    3    4    5    6 
5120 5188 5538 4416 5036 5592 

In [50]:
Notas_Victoria_primaria$Año_curso <- str_replace(Notas_Victoria_primaria$Año_curso, "1" , "PRIMERO")
Notas_Victoria_primaria$Año_curso <- str_replace(Notas_Victoria_primaria$Año_curso, "2" , "SEGUNDO")
Notas_Victoria_primaria$Año_curso <- str_replace(Notas_Victoria_primaria$Año_curso, "3" , "TERCERO")
Notas_Victoria_primaria$Año_curso <- str_replace(Notas_Victoria_primaria$Año_curso, "4" , "CUARTO")
Notas_Victoria_primaria$Año_curso <- str_replace(Notas_Victoria_primaria$Año_curso, "5" , "QUINTO")
Notas_Victoria_primaria$Año_curso <- str_replace(Notas_Victoria_primaria$Año_curso, "6" , "SEXTO")

In [51]:
Notas_Victoria_1Grado <- distinct(filter(Notas_Victoria_primaria, Año_curso == "PRIMERO"))
math <- Notas_Victoria_1Grado %>% group_by(CodigoUnicoEscolar,asignatura) %>%  filter(asignatura=="MATEMATICA")
math
tabla1 <- Notas_Victoria_1Grado %>% group_by(asignatura) %>%  summarise(conteo=n())
tabla1

CodigoUnicoEscolar,departamento,localidad,EsPrivada,turno,curso,periodoEvaluatorio,Documento,asignatura,idalumno,...,IdModEnseñanza,Modalidad,ModEnseñanza,año,NotasPrimerT,NotasSegundoT,NotasTercerT,Promedio,Condición,Año_curso
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<int>,...,<int>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
9005397,VICTORIA,VICTORIA,N,COMPLETO,1 - B Prim,Tercer Trimestre,54711442,MATEMATICA,547730,...,5,Primaria,Comun,2023,7,5,5,5.67,Rinde,PRIMERO
9000756,VICTORIA,VICTORIA,N,TARDE,1 - A Prim,Tercer Trimestre,55049796,MATEMATICA,838057,...,15,Primaria,Escuelas NINA,2023,7,7,6,6.67,Aprobado,PRIMERO
9000756,VICTORIA,VICTORIA,N,TARDE,1 - A Prim,Tercer Trimestre,55049851,MATEMATICA,836148,...,15,Primaria,Escuelas NINA,2023,6,6,6,6.00,Aprobado,PRIMERO
9005406,VICTORIA,VICTORIA,N,TARDE,1 - D Prim,Tercer Trimestre,55117447,MATEMATICA,832109,...,5,Primaria,Comun,2023,6,6,5,5.67,Rinde,PRIMERO
9005394,VICTORIA,VICTORIA,N,COMPLETO,1 - B Prim,Tercer Trimestre,55182685,MATEMATICA,901026,...,5,Primaria,Comun,2023,5,5,6,5.33,Rinde,PRIMERO
9005397,VICTORIA,VICTORIA,N,COMPLETO,1 - B Prim,Tercer Trimestre,55317870,MATEMATICA,648903,...,5,Primaria,Comun,2023,5,5,5,5.00,Rinde,PRIMERO
9005406,VICTORIA,VICTORIA,N,TARDE,1 - C Prim,Tercer Trimestre,55318313,MATEMATICA,777763,...,5,Primaria,Comun,2023,6,5,5,5.33,Rinde,PRIMERO
9005394,VICTORIA,VICTORIA,N,COMPLETO,1 - C Prim,Tercer Trimestre,55404069,MATEMATICA,901006,...,5,Primaria,Comun,2023,7,8,8,7.67,Aprobado,PRIMERO
9004017,VICTORIA,VICTORIA,N,TARDE,1 - B Prim,Tercer Trimestre,55405424,MATEMATICA,1018722,...,5,Primaria,Comun,2023,7,6,5,6.00,Aprobado,PRIMERO


asignatura,conteo
<chr>,<int>
ARTES VISUALES,580
CATEQUESIS,61
CIENCIAS NATURALES,573
CIENCIAS SOCIALES,542
EDI,72
EDUCACION FISICA,574
EDUCACION MUSICAL,546
EDUCACION TECNOLOGICA,564
INFORMATICA,43


In [52]:
Notas_Victoria_2Grado <- distinct(filter(Notas_Victoria_primaria, Año_curso == "SEGUNDO"))
math <- Notas_Victoria_2Grado %>% group_by(CodigoUnicoEscolar,asignatura) %>%  filter(asignatura=="MATEMATICA")
math
tabla2 <- Notas_Victoria_2Grado %>% group_by(asignatura) %>%  summarise(conteo=n())
tabla2

CodigoUnicoEscolar,departamento,localidad,EsPrivada,turno,curso,periodoEvaluatorio,Documento,asignatura,idalumno,...,IdModEnseñanza,Modalidad,ModEnseñanza,año,NotasPrimerT,NotasSegundoT,NotasTercerT,Promedio,Condición,Año_curso
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<int>,...,<int>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
9005394,VICTORIA,VICTORIA,N,COMPLETO,2 - C Prim,Tercer Trimestre,53810677,MATEMATICA,550253,...,5,Primaria,Comun,2023,5,5,5,5.00,Rinde,SEGUNDO
9000291,VICTORIA,VICTORIA,N,TARDE,2 - A Prim,Tercer Trimestre,53837774,MATEMATICA,796967,...,5,Primaria,Comun,2023,6,6,7,6.33,Aprobado,SEGUNDO
9005394,VICTORIA,VICTORIA,N,COMPLETO,2 - C Prim,Tercer Trimestre,53839289,MATEMATICA,732053,...,5,Primaria,Comun,2023,6,6,7,6.33,Aprobado,SEGUNDO
9005394,VICTORIA,VICTORIA,N,COMPLETO,2 - C Prim,Tercer Trimestre,53839321,MATEMATICA,745368,...,5,Primaria,Comun,2023,6,6,6,6.00,Aprobado,SEGUNDO
9000756,VICTORIA,VICTORIA,N,TARDE,2 - A Prim,Tercer Trimestre,54006777,MATEMATICA,800648,...,15,Primaria,Escuelas NINA,2023,6,6,5,5.67,Rinde,SEGUNDO
9005397,VICTORIA,VICTORIA,N,COMPLETO,2 - A Prim,Tercer Trimestre,54006820,MATEMATICA,773645,...,5,Primaria,Comun,2023,7,7,7,7.00,Aprobado,SEGUNDO
9004017,VICTORIA,VICTORIA,N,TARDE,2 - D Prim,Tercer Trimestre,54115899,MATEMATICA,618700,...,5,Primaria,Comun,2023,6,5,5,5.33,Rinde,SEGUNDO
9000291,VICTORIA,VICTORIA,N,TARDE,2 - A Prim,Tercer Trimestre,54119842,MATEMATICA,1092942,...,5,Primaria,Comun,2023,6,5,5,5.33,Rinde,SEGUNDO
9000759,VICTORIA,VICTORIA,N,TARDE,2 - D Prim,Tercer Trimestre,54152889,MATEMATICA,648866,...,5,Primaria,Comun,2023,6,6,6,6.00,Aprobado,SEGUNDO


asignatura,conteo
<chr>,<int>
ACOMPAÑAMIENTO AL ESTUDIO,16
ARTES VISUALES,593
CATEQUESIS,60
CIENCIAS NATURALES,560
CIENCIAS SOCIALES,531
EDI,80
EDUCACION FISICA,567
EDUCACION MUSICAL,518
EDUCACION TECNOLOGICA,528


In [53]:
Notas_Victoria_3Grado <- distinct(filter(Notas_Victoria_primaria, Año_curso == "TERCERO"))
math <- Notas_Victoria_3Grado %>% group_by(CodigoUnicoEscolar,asignatura) %>%  filter(asignatura=="MATEMATICA")
math
tabla3 <- Notas_Victoria_3Grado %>% group_by(asignatura) %>%  summarise(conteo=n())
tabla3

CodigoUnicoEscolar,departamento,localidad,EsPrivada,turno,curso,periodoEvaluatorio,Documento,asignatura,idalumno,...,IdModEnseñanza,Modalidad,ModEnseñanza,año,NotasPrimerT,NotasSegundoT,NotasTercerT,Promedio,Condición,Año_curso
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<int>,...,<int>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
9005397,VICTORIA,VICTORIA,N,COMPLETO,3 - A Prim,Tercer Trimestre,53110927,MATEMATICA,614367,...,5,Primaria,Comun,2023,7,7,7,7.00,Aprobado,TERCERO
9004017,VICTORIA,VICTORIA,N,TARDE,3 - D Prim,Tercer Trimestre,53250964,MATEMATICA,665786,...,5,Primaria,Comun,2023,8,8,8,8.00,Aprobado,TERCERO
9000759,VICTORIA,VICTORIA,N,TARDE,3 - D Prim,Tercer Trimestre,53251738,MATEMATICA,646724,...,5,Primaria,Comun,2023,6,6,6,6.00,Aprobado,TERCERO
9004017,VICTORIA,VICTORIA,N,TARDE,3 - B Prim,Tercer Trimestre,53251757,MATEMATICA,609153,...,5,Primaria,Comun,2023,7,7,7,7.00,Aprobado,TERCERO
9005397,VICTORIA,VICTORIA,N,COMPLETO,3 - B Prim,Tercer Trimestre,53251759,MATEMATICA,492445,...,5,Primaria,Comun,2023,7,9,9,8.33,Aprobado,TERCERO
9004020,VICTORIA,VICTORIA,N,MAÑANA,3 - A Prim,Tercer Trimestre,53271226,MATEMATICA,737323,...,5,Primaria,Comun,2023,8,9,10,9.00,Aprobado,TERCERO
9005397,VICTORIA,VICTORIA,N,COMPLETO,3 - A Prim,Tercer Trimestre,53271349,MATEMATICA,911923,...,5,Primaria,Comun,2023,7,8,8,7.67,Aprobado,TERCERO
9005397,VICTORIA,VICTORIA,N,COMPLETO,3 - B Prim,Tercer Trimestre,53359641,MATEMATICA,698713,...,5,Primaria,Comun,2023,8,9,10,9.00,Aprobado,TERCERO
9001791,VICTORIA,VICTORIA,N,MAÑANA,3 - A Prim,Tercer Trimestre,53359658,MATEMATICA,688888,...,5,Primaria,Comun,2023,6,6,7,6.33,Aprobado,TERCERO


asignatura,conteo
<chr>,<int>
ARTES VISUALES,594
CATEQUESIS,72
CIENCIAS NATURALES,584
CIENCIAS SOCIALES,555
EDI,81
EDUCACION FISICA,568
EDUCACION MUSICAL,562
EDUCACION TECNOLOGICA,541
INFORMATICA,46


In [54]:
Notas_Victoria_4Grado <- distinct(filter(Notas_Victoria_primaria, Año_curso == "CUARTO"))
math <- Notas_Victoria_4Grado %>% group_by(CodigoUnicoEscolar,asignatura) %>%  filter(asignatura=="MATEMATICA")
math
tabla4 <- Notas_Victoria_4Grado %>% group_by(asignatura) %>%  summarise(conteo=n())
tabla4

CodigoUnicoEscolar,departamento,localidad,EsPrivada,turno,curso,periodoEvaluatorio,Documento,asignatura,idalumno,...,IdModEnseñanza,Modalidad,ModEnseñanza,año,NotasPrimerT,NotasSegundoT,NotasTercerT,Promedio,Condición,Año_curso
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<int>,...,<int>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
9000756,VICTORIA,VICTORIA,N,MAÑANA,4 - A Prim,Tercer Trimestre,50746843,MATEMATICA,512733,...,15,Primaria,Escuelas NINA,2023,5,6,7,6.00,Aprobado,CUARTO
9000753,VICTORIA,VICTORIA,N,MAÑANA,4 - A Prim,Tercer Trimestre,51248164,MATEMATICA,484567,...,5,Primaria,Comun,2023,6,6,6,6.00,Aprobado,CUARTO
9004020,VICTORIA,VICTORIA,N,TARDE,4 - A Prim,Tercer Trimestre,51503751,MATEMATICA,737328,...,5,Primaria,Comun,2023,8,8,9,8.33,Aprobado,CUARTO
9004017,VICTORIA,VICTORIA,N,TARDE,4 - B Prim,Tercer Trimestre,5223891,MATEMATICA,551929,...,5,Primaria,Comun,2023,6,6,5,5.67,Rinde,CUARTO
9004017,VICTORIA,VICTORIA,N,TARDE,4 - D Prim,Tercer Trimestre,52254142,MATEMATICA,495320,...,5,Primaria,Comun,2023,8,8,8,8.00,Aprobado,CUARTO
9005397,VICTORIA,VICTORIA,N,COMPLETO,4 - B Prim,Tercer Trimestre,52255093,MATEMATICA,584989,...,5,Primaria,Comun,2023,5,5,5,5.00,Rinde,CUARTO
9000753,VICTORIA,VICTORIA,N,MAÑANA,4 - A Prim,Tercer Trimestre,52258402,MATEMATICA,549742,...,5,Primaria,Comun,2023,5,5,5,5.00,Rinde,CUARTO
9004017,VICTORIA,VICTORIA,N,TARDE,4 - D Prim,Tercer Trimestre,52258415,MATEMATICA,421298,...,5,Primaria,Comun,2023,8,8,5,7.00,Aprobado,CUARTO
9004020,VICTORIA,VICTORIA,N,TARDE,4 - A Prim,Tercer Trimestre,52258426,MATEMATICA,421278,...,5,Primaria,Comun,2023,7,7,8,7.33,Aprobado,CUARTO


asignatura,conteo
<chr>,<int>
ARTES VISUALES,440
CATEQUESIS,63
CIENCIAS NATURALES,479
CIENCIAS SOCIALES,436
EDI,64
EDUCACION FISICA,452
EDUCACION MUSICAL,438
EDUCACION TECNOLOGICA,422
INGLES,201


In [55]:
Notas_Victoria_5Grado <- distinct(filter(Notas_Victoria_primaria, Año_curso == "QUINTO"))
math <- Notas_Victoria_5Grado %>% group_by(CodigoUnicoEscolar,asignatura) %>%  filter(asignatura=="MATEMATICA")
math
tabla5 <- Notas_Victoria_5Grado %>% group_by(asignatura) %>%  summarise(conteo=n())
tabla5

CodigoUnicoEscolar,departamento,localidad,EsPrivada,turno,curso,periodoEvaluatorio,Documento,asignatura,idalumno,...,IdModEnseñanza,Modalidad,ModEnseñanza,año,NotasPrimerT,NotasSegundoT,NotasTercerT,Promedio,Condición,Año_curso
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<int>,...,<int>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
9005394,VICTORIA,VICTORIA,N,COMPLETO,5 - A Prim,Tercer Trimestre,49538621,MATEMATICA,902818,...,5,Primaria,Comun,2023,6,6,6,6.00,Aprobado,QUINTO
9004020,VICTORIA,VICTORIA,N,TARDE,5 - A Prim,Tercer Trimestre,49852386,MATEMATICA,737322,...,5,Primaria,Comun,2023,7,7,6,6.67,Aprobado,QUINTO
9005397,VICTORIA,VICTORIA,N,COMPLETO,5 - B Prim,Tercer Trimestre,50285401,MATEMATICA,503557,...,5,Primaria,Comun,2023,7,5,5,5.67,Rinde,QUINTO
9004020,VICTORIA,VICTORIA,N,TARDE,5 - A Prim,Tercer Trimestre,50491504,MATEMATICA,725546,...,5,Primaria,Comun,2023,8,8,9,8.33,Aprobado,QUINTO
9004020,VICTORIA,VICTORIA,N,TARDE,5 - A Prim,Tercer Trimestre,50491504,MATEMATICA,725546,...,5,Primaria,Comun,2023,9,9,10,9.33,Aprobado,QUINTO
9004020,VICTORIA,VICTORIA,N,TARDE,5 - A Prim,Tercer Trimestre,50587517,MATEMATICA,542054,...,5,Primaria,Comun,2023,8,6,7,7.00,Aprobado,QUINTO
9004020,VICTORIA,VICTORIA,N,TARDE,5 - A Prim,Tercer Trimestre,50587517,MATEMATICA,542054,...,5,Primaria,Comun,2023,7,7,6,6.67,Aprobado,QUINTO
9000756,VICTORIA,VICTORIA,N,MAÑANA,5 - A Prim,Tercer Trimestre,50588032,MATEMATICA,631702,...,15,Primaria,Escuelas NINA,2023,6,6,6,6.00,Aprobado,QUINTO
9005394,VICTORIA,VICTORIA,N,COMPLETO,5 - A Prim,Tercer Trimestre,50651213,MATEMATICA,921601,...,5,Primaria,Comun,2023,6,5,5,5.33,Rinde,QUINTO


asignatura,conteo
<chr>,<int>
AC.EN MATEMATICA Y CS. NATURALES,18
ARTES VISUALES,507
CATEQUESIS,63
CIENCIAS NATURALES,518
CIENCIAS SOCIALES,510
EDI,59
EDUCACION FISICA,504
EDUCACION MUSICAL,486
EDUCACION TECNOLOGICA,479


In [56]:
Notas_Victoria_6Grado <- distinct(filter(Notas_Victoria_primaria, Año_curso == "SEXTO"))
math <- Notas_Victoria_6Grado %>% group_by(CodigoUnicoEscolar,asignatura) %>%  filter(asignatura=="MATEMATICA")
math
tabla6 <- Notas_Victoria_6Grado %>% group_by(asignatura) %>%  summarise(conteo=n())
tabla6

CodigoUnicoEscolar,departamento,localidad,EsPrivada,turno,curso,periodoEvaluatorio,Documento,asignatura,idalumno,...,IdModEnseñanza,Modalidad,ModEnseñanza,año,NotasPrimerT,NotasSegundoT,NotasTercerT,Promedio,Condición,Año_curso
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<int>,...,<int>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
9000753,VICTORIA,VICTORIA,N,MAÑANA,6 - B Prim,Tercer Trimestre,49438652,MATEMATICA,512667,...,5,Primaria,Comun,2023,6,6,7,6.33,Aprobado,SEXTO
9005394,VICTORIA,VICTORIA,N,COMPLETO,6 - A Prim,Tercer Trimestre,49858717,MATEMATICA,598269,...,5,Primaria,Comun,2023,5,5,5,5.00,Rinde,SEXTO
9004020,VICTORIA,VICTORIA,N,TARDE,6 - A Prim,Tercer Trimestre,49858733,MATEMATICA,725259,...,5,Primaria,Comun,2023,7,7,7,7.00,Aprobado,SEXTO
9004020,VICTORIA,VICTORIA,N,TARDE,6 - A Prim,Tercer Trimestre,49858733,MATEMATICA,725259,...,5,Primaria,Comun,2023,8,7,6,7.00,Aprobado,SEXTO
9000759,VICTORIA,VICTORIA,N,TARDE,6 - D Prim,Tercer Trimestre,50069559,MATEMATICA,512978,...,5,Primaria,Comun,2023,6,6,5,5.67,Rinde,SEXTO
9006249,VICTORIA,VICTORIA,S,MAÑANA,6 - A Prim,Tercer Trimestre,50202892,MATEMATICA,559737,...,5,Primaria,Comun,2023,8,8,8,8.00,Aprobado,SEXTO
9000753,VICTORIA,VICTORIA,N,MAÑANA,6 - B Prim,Tercer Trimestre,50285393,MATEMATICA,412377,...,5,Primaria,Comun,2023,6,6,6,6.00,Aprobado,SEXTO
9005394,VICTORIA,VICTORIA,N,COMPLETO,6 - A Prim,Tercer Trimestre,50285457,MATEMATICA,512028,...,5,Primaria,Comun,2023,8,5,6,6.33,Aprobado,SEXTO
9004017,VICTORIA,VICTORIA,N,TARDE,6 - D Prim,Tercer Trimestre,50310485,MATEMATICA,1018628,...,5,Primaria,Comun,2023,6,5,5,5.33,Rinde,SEXTO


asignatura,conteo
<chr>,<int>
AC.EN MATEMATICA Y CS. NATURALES,23
ARTES VISUALES,509
CATEQUESIS,61
CIENCIAS NATURALES,515
CIENCIAS SOCIALES,509
EDI,71
EDUCACION FISICA,529
EDUCACION MUSICAL,511
EDUCACION TECNOLOGICA,506


In [57]:
dim(Notas_Victoria_primaria)

[1] 30890    27

In [31]:
write_xlsx(Notas_Victoria_primaria, "C:/Users/ayrto/OneDrive/Escritorio/Exploración de Datos Multivariados/Trabajo Practico/VICTORIA_BASE_PRIMARIA_LIMPIA.xlsx")

ERROR: Error: Error in libxlsxwriter: 'Error creating output xlsx file. Usually a permissions error.'


In [58]:
PrimerT_secundaria <- filter(Victoria_secundaria, periodoEvaluatorio == "Primer Trimestre")
PrimerT_secundaria <- rename(PrimerT_secundaria,NotasPrimerT = nota)
SegundoT_secundaria <- filter(Victoria_secundaria, periodoEvaluatorio == "Segundo Trimestre")
SegundoT_secundaria <- rename(SegundoT_secundaria,NotasSegundoT = nota)
TercerT_secundaria <- filter(Victoria_secundaria, periodoEvaluatorio == "Tercer Trimestre")
TercerT_secundaria <- rename(TercerT_secundaria,NotasTercerT = nota)

In [59]:
aux2 <- merge(x = PrimerT_secundaria, y = SegundoT_secundaria, by = c("Documento", "asignatura"))
auxiliar2 <- merge(x = aux2, y = TercerT_secundaria, by = c("Documento", "asignatura"))

In [60]:
Notas_Victoria_secundaria <- select(auxiliar2, 'CodigoUnicoEscolar', 'departamento', 'localidad', 'EsPrivada', 'turno' , 'curso' , 'periodoEvaluatorio' , 'Documento', 'asignatura','idalumno', 'observaciones', 'idSubOrganizacion', 'idDivision', 'NivelEnseñanza', 'orden','Nivel', 'esMultiaño', 'IdModEnseñanza', 'Modalidad', 'ModEnseñanza', 'año','NotasPrimerT','NotasSegundoT', 'NotasTercerT')

In [61]:
Notas_Victoria_secundaria$Promedio <-round ((as.numeric(Notas_Victoria_secundaria$NotasPrimerT)+as.numeric(Notas_Victoria_secundaria$NotasSegundoT)+as.numeric(Notas_Victoria_secundaria$NotasTercerT))/3,2)
head(Notas_Victoria_secundaria)

,CodigoUnicoEscolar,departamento,localidad,EsPrivada,turno,curso,periodoEvaluatorio,Documento,asignatura,idalumno,...,Nivel,esMultiaño,IdModEnseñanza,Modalidad,ModEnseñanza,año,NotasPrimerT,NotasSegundoT,NotasTercerT,Promedio
,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<int>,...,<int>,<chr>,<int>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
1,9008373,VICTORIA,VICTORIA,N,NOCHE,3 - B 4000A,Tercer Trimestre,37098670,TECNOLOGIA DE LA INFORMACION Y LA COMUNICACION,1157557,...,3,N,5,Secundaria,Comun,2023,3,3,1,2.33
2,9008373,VICTORIA,VICTORIA,N,NOCHE,3 - B 4000A,Tercer Trimestre,37098670,TUTORIAS,1157557,...,3,N,5,Secundaria,Comun,2023,2,2,2,2.00
3,9008373,VICTORIA,VICTORIA,N,NOCHE,3 - B 4000A,Tercer Trimestre,38052214,HISTORIA,907312,...,3,N,5,Secundaria,Comun,2023,9,9,9,9.00
4,9008373,VICTORIA,VICTORIA,N,NOCHE,3 - B 4000A,Tercer Trimestre,38052214,LENGUA Y LITERATURA,907312,...,3,N,5,Secundaria,Comun,2023,9,7,10,8.67
5,9008373,VICTORIA,VICTORIA,N,NOCHE,3 - B 4000A,Tercer Trimestre,38052214,LENGUAJE Y PRODUCCION MUSICAL II,907312,...,3,N,5,Secundaria,Comun,2023,10,10,10,10.00
6,9008373,VICTORIA,VICTORIA,N,NOCHE,3 - B 4000A,Tercer Trimestre,38052214,LENGUAJE Y PRODUCCION VISUAL II,907312,...,3,N,5,Secundaria,Comun,2023,8,9,8,8.33


In [62]:
Notas_Victoria_secundaria <- Notas_Victoria_secundaria %>% mutate(Condición = case_when(
  Promedio >= 6 ~ "Aprobado",
  Promedio <=6  ~ "Rinde",
   TRUE~ "sin registro completo"))
head(Notas_Victoria_secundaria)

,CodigoUnicoEscolar,departamento,localidad,EsPrivada,turno,curso,periodoEvaluatorio,Documento,asignatura,idalumno,...,esMultiaño,IdModEnseñanza,Modalidad,ModEnseñanza,año,NotasPrimerT,NotasSegundoT,NotasTercerT,Promedio,Condición
,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<int>,...,<chr>,<int>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,9008373,VICTORIA,VICTORIA,N,NOCHE,3 - B 4000A,Tercer Trimestre,37098670,TECNOLOGIA DE LA INFORMACION Y LA COMUNICACION,1157557,...,N,5,Secundaria,Comun,2023,3,3,1,2.33,Rinde
2,9008373,VICTORIA,VICTORIA,N,NOCHE,3 - B 4000A,Tercer Trimestre,37098670,TUTORIAS,1157557,...,N,5,Secundaria,Comun,2023,2,2,2,2.00,Rinde
3,9008373,VICTORIA,VICTORIA,N,NOCHE,3 - B 4000A,Tercer Trimestre,38052214,HISTORIA,907312,...,N,5,Secundaria,Comun,2023,9,9,9,9.00,Aprobado
4,9008373,VICTORIA,VICTORIA,N,NOCHE,3 - B 4000A,Tercer Trimestre,38052214,LENGUA Y LITERATURA,907312,...,N,5,Secundaria,Comun,2023,9,7,10,8.67,Aprobado
5,9008373,VICTORIA,VICTORIA,N,NOCHE,3 - B 4000A,Tercer Trimestre,38052214,LENGUAJE Y PRODUCCION MUSICAL II,907312,...,N,5,Secundaria,Comun,2023,10,10,10,10.00,Aprobado
6,9008373,VICTORIA,VICTORIA,N,NOCHE,3 - B 4000A,Tercer Trimestre,38052214,LENGUAJE Y PRODUCCION VISUAL II,907312,...,N,5,Secundaria,Comun,2023,8,9,8,8.33,Aprobado


In [63]:
table(Notas_Victoria_secundaria$asignatura)
head(Notas_Victoria_secundaria$asignatura,10)
Notas_Victoria_secundaria$asignatura <- toupper(Notas_Victoria_secundaria$asignatura)
head(Notas_Victoria_secundaria$asignatura,10)


                                       ANALISIS MATEMATICO 
                                                        43 
                                            ARTES VISUALES 
                                                      2443 
                                                AUTO C.A.D 
                                                        24 
                                        BIDIMENSION I Y II 
                                                        51 
                                                  BIOLOGIA 
                                                      2972 
                                                     C.A.D 
                                                        16 
                                                CATEQUESIS 
                                                       127 
                                    CEREALES Y OLEAGINOSAS 
                                                        21 
                             CIENCIAS N

[1] "TECNOLOGIA DE LA INFORMACION Y LA COMUNICACION" 
 [2] "TUTORIAS"                                       
 [3] "HISTORIA"                                       
 [4] "LENGUA Y LITERATURA"                            
 [5] "LENGUAJE Y PRODUCCION MUSICAL II"               
 [6] "LENGUAJE Y PRODUCCION VISUAL II"                
 [7] "LENGUAS EXTRANJERAS"                            
 [8] "MATEMATICA"                                     
 [9] "PRODUCCION MULTIMEDIAL EN LENGUAJES ART. I y II"
[10] "PRODUCCIONES ARTISTICAS INTEGRADAS"

[1] "TECNOLOGIA DE LA INFORMACION Y LA COMUNICACION" 
 [2] "TUTORIAS"                                       
 [3] "HISTORIA"                                       
 [4] "LENGUA Y LITERATURA"                            
 [5] "LENGUAJE Y PRODUCCION MUSICAL II"               
 [6] "LENGUAJE Y PRODUCCION VISUAL II"                
 [7] "LENGUAS EXTRANJERAS"                            
 [8] "MATEMATICA"                                     
 [9] "PRODUCCION MULTIMEDIAL EN LENGUAJES ART. I Y II"
[10] "PRODUCCIONES ARTISTICAS INTEGRADAS"

In [64]:
Notas_Victoria_secundaria$asignatura <- str_replace_all(Notas_Victoria_secundaria$asignatura, ".*BIOLOGIA.*", "BIOLOGIA")
Notas_Victoria_secundaria$asignatura <- str_replace_all(Notas_Victoria_secundaria$asignatura, ".*MATEMATICA*", "MATEMATICA")
Notas_Victoria_secundaria$asignatura <- str_replace_all(Notas_Victoria_secundaria$asignatura, ".*LENGUA*", "LENGUA")
Notas_Victoria_secundaria$asignatura <- str_replace_all(Notas_Victoria_secundaria$asignatura, ".*EXTRANJERA*", "INGLES")
Notas_Victoria_secundaria$asignatura <- str_replace_all(Notas_Victoria_secundaria$asignatura, ".*GEOGRAFIA.*", "GEOGRAFIA")
Notas_Victoria_secundaria$asignatura <- str_replace_all(Notas_Victoria_secundaria$asignatura, ".*HISTORIA*", "HISTORIA")
Notas_Victoria_secundaria$asignatura <- str_replace_all(Notas_Victoria_secundaria$asignatura, ".*BIOLOGIA.*", "BIOLOGIA")
Notas_Victoria_secundaria$asignatura <- str_replace_all(Notas_Victoria_secundaria$asignatura, ".*ETICA*", "ETICA")
Notas_Victoria_secundaria$asignatura <- str_replace_all(Notas_Victoria_secundaria$asignatura, ".*MUSICA*", "MUSICA")
Notas_Victoria_secundaria$asignatura <- str_replace_all(Notas_Victoria_secundaria$asignatura, ".*ETICA*", "ETICA")
Notas_Victoria_secundaria$asignatura <- str_replace_all(Notas_Victoria_secundaria$asignatura, ".*FILOSOFIA*", "FILOSOFIA")
Notas_Victoria_secundaria$asignatura <- str_replace_all(Notas_Victoria_secundaria$asignatura, ".*ECONOMIA*", "ECONOMIA")
Notas_Victoria_secundaria$asignatura <- str_replace_all(Notas_Victoria_secundaria$asignatura, ".*FISICO QUIMICA*", "FISICA Y QUIMICA")
Notas_Victoria_secundaria$asignatura <- gsub("AUTO C.A.D", "C.A.D", Notas_Victoria_secundaria$asignatura)
Notas_Victoria_secundaria$asignatura <- gsub("DERECHO COMERCIAL", "DERECHO", Notas_Victoria_secundaria$asignatura)
Notas_Victoria_secundaria$asignatura <- gsub("CEREALES Y OLEAGINOSAS|MARCO JURIDICO DE LOS CIENCIAS AGROPECUARIAS |PRODUCCION HORTICOLA|PRODUCCION DE PLANTAS EN VIVERO |PRODUCCION DE FORRAJES|PRODUCCION DE CERDOS|PRODUCCION DE CAPRINOS|PRODUCCION BOVINO PARA CARNE|PROCESOS PRODUCTIVOS|INSTALACIONES AGROPECUARIAS|CIENCIAS NATURALES APLICADAS A PRO. AGROPECUARIOS|INDUSTRIA DE PROD. LACTEOS DE BOVINO O CAPRINO|INDUSTRIA CARNE PORCINA", "CIENCIAS AGROPECUARIAS", Notas_Victoria_secundaria$asignatura)
Notas_Victoria_secundaria$asignatura <- gsub("CIENCIAS NATURALES (FISICO QUIMICA)|FISICO Y QUIMICA)|CIENCIAS NATURALES FISICO-QUIMICA|FISICA Y QUIMICA|ESPACIO DE LA ORIENTACION EN CIENCIAS NATURALES (FISICO QUIMICA)", "FISICO Y QUIMICA", Notas_Victoria_secundaria$asignatura)
Notas_Victoria_secundaria$asignatura <- gsub("CIRCUITOS TURISTICOS I Y II|MARKETING TURISTICO|INTRODUCCION AL TURISMO|EMPRESAS TURISTICAS|INFRAESTRUCTURA TURISTICA Y TURISMO TERMAL II|INFRAESTRUCTURA TURISTICA Y TURISMO TERMAL I|INFRAESTRUCTURA TURISTICA Y TURISMO TERMAL|CIRCUITOS TURISTICOS II|DESARROLLO DE LOCALIDADES TURISTICAS", "TURISMO", Notas_Victoria_secundaria$asignatura)
Notas_Victoria_secundaria$asignatura <- gsub("ECONOMIA II", "ECONOMIA", Notas_Victoria_secundaria$asignatura)
Notas_Victoria_secundaria$asignatura <- gsub("ETICA Y CIUDADANA|ETICAS CONTEMPORANEAS I", "ETICA", Notas_Victoria_secundaria$asignatura)
Notas_Victoria_secundaria$asignatura <- gsub("FILOSOFIA II", "FILOSOFIA", Notas_Victoria_secundaria$asignatura)
Notas_Victoria_secundaria$asignatura <- gsub("FISICA APLICADA|TRABAJOS PRACTICOS DE FISICA APLICADA", "FISICA", Notas_Victoria_secundaria$asignatura)
Notas_Victoria_secundaria$asignatura <- gsub("FRANCÉS", "FRANCES", Notas_Victoria_secundaria$asignatura)
Notas_Victoria_secundaria$asignatura <- gsub("INGLES (INGLES)|INGLES (INGLES)|INGLESS", "INGLES", Notas_Victoria_secundaria$asignatura)
Notas_Victoria_secundaria$asignatura <- gsub("PORTUGUÉS", "PORTUGUES", Notas_Victoria_secundaria$asignatura)
Notas_Victoria_secundaria$asignatura <- gsub("QUIMICA ANALITICA CUALITATIVA|QUIMICA APLICADA|TRABAJOS PRACTICOS DE QUIMICA ORGANICA II|TRABAJOS PRACTICOS DE QUIMICA ORGANICA |TRABAJOS PRACTICOS DE QUIMICA INORGANICA |TRABAJOS PRACTICOS DE QUIMICA INDUSTRIAL APLICADA |TRABAJOS PRACTICOS DE QUIMICA INDUSTRIAL|TRABAJOS PRACTICOS DE QUIMICA GENERAL|TRABAJOS PRACTICOS DE QUIMICA ANALITICA CUANTIT.|TRABAJOS PRACTICOS DE PROC. Y OPERACIONES QUIMICAS|TRABAJOS PRACTICOS DE PROC. Y OPERACIONES QUIMICAS|QUIMICA ANALITICA CUANTITATIVA|QUIMICA GENERAL|QUIMICA INORGANICA|QUIMICA ORGANICA II|TALLER MODULAR QUIMICA", "QUIMICA", Notas_Victoria_secundaria$asignatura)
Notas_Victoria_secundaria$asignatura <- gsub("TECNOLOGIA I|TECNOLOGIA II|TECNOLOGIA III|EDUCACION TECNOLOGICA", "TECNOLOGIA", Notas_Victoria_secundaria$asignatura)
Notas_Victoria_secundaria$asignatura <- gsub("TUTORIAS DISCIPLINARES", "TUTORIAS", Notas_Victoria_secundaria$asignatura)
Notas_Victoria_secundaria$asignatura <- gsub("GESTION DE LA PRODUCCION ARTISTICA", "ARTES VISUALES", Notas_Victoria_secundaria$asignatura)
Notas_Victoria_secundaria$asignatura <- gsub("HISTORIA)", "HISTORIA", Notas_Victoria_secundaria$asignatura)
Notas_Victoria_secundaria$asignatura <- gsub("MATEMATICA|MATEMATICAO", "MATEMATICA", Notas_Victoria_secundaria$asignatura)
Notas_Victoria_secundaria$asignatura <- gsub("LENGUA Y LITERATURA|LITERATURA ARGENTINA|LITERATURA LATINOAMERICANA", "LENGUA Y LITERATURA", Notas_Victoria_secundaria$asignatura)
Notas_Victoria_secundaria$asignatura <- gsub("INGLES\\s*\\(INGLES\\)", "INGLES", Notas_Victoria_secundaria$asignatura)

In [65]:
table(Notas_Victoria_secundaria$asignatura)


                                            ARTES VISUALES 
                                                      2473 
                                        BIDIMENSION I Y II 
                                                        51 
                                                  BIOLOGIA 
                                                      3495 
                                                     C.A.D 
                                                        40 
                                                CATEQUESIS 
                                                       127 
                                    CIENCIAS AGROPECUARIAS 
                                                       185 
                                              COMUNICACION 
                                                        15 
              COMUNICACION, EDUCACION Y NUEVAS TECNOLOGIAS 
                                                         6 
                                    CUL

In [66]:
table(Notas_Victoria_secundaria$curso)
Notas_Victoria_secundaria <- Notas_Victoria_secundaria[Notas_Victoria_secundaria$curso != "CN - RURAL CNR", ]
table(Notas_Victoria_secundaria$curso)


                  1 - 1 CBC                1 - 1 RCBCET 
                        837                         295 
                  1 - 2 CBC                1 - 2 RCBCET 
                        568                         370 
                  1 - 3 CBC                1 - 3 RCBCET 
                        463                         319 
                  1 - 4 CBC                1 - 4 RCBCET 
                        630                         310 
                  1 - 5 CBC                1 - 5 RCBEAT 
                        195                         700 
                1 - A 4000E                   1 - A ART 
                         20                         360 
                  1 - A CBC 1 - A Cs. Soc y Hum 485/10C 
                       1714                           1 
                1 - B 4000A                   1 - B CBC 
                        154                         394 
                1 - C 4000A                   1 - C CBC 
                        102   


                  1 - 1 CBC                1 - 1 RCBCET 
                        837                         295 
                  1 - 2 CBC                1 - 2 RCBCET 
                        568                         370 
                  1 - 3 CBC                1 - 3 RCBCET 
                        463                         319 
                  1 - 4 CBC                1 - 4 RCBCET 
                        630                         310 
                  1 - 5 CBC                1 - 5 RCBEAT 
                        195                         700 
                1 - A 4000E                   1 - A ART 
                         20                         360 
                  1 - A CBC 1 - A Cs. Soc y Hum 485/10C 
                       1714                           1 
                1 - B 4000A                   1 - B CBC 
                        154                         394 
                1 - C 4000A                   1 - C CBC 
                        102   

In [67]:
Notas_Victoria_secundaria <- Notas_Victoria_secundaria[Notas_Victoria_secundaria$Condición != "sin registro completo", ]

In [68]:
Notas_Victoria_secundaria$curso <- as.character(Notas_Victoria_secundaria$curso)
Notas_Victoria_secundaria$Año_curso <- str_sub(Notas_Victoria_secundaria$curso,1, 1)
print(head(Notas_Victoria_secundaria$Año_curso),2)
table(Notas_Victoria_secundaria$Año_curso)

[1] "3" "3" "3" "3" "3" "3"



    1     2     3     4     5     6     7 
 7845 10177  8298  8228  7567  8657   819 

In [69]:
Notas_Victoria_secundaria$Año_curso <- str_replace(Notas_Victoria_secundaria$Año_curso, "1" , "PRIMERO")
Notas_Victoria_secundaria$Año_curso <- str_replace(Notas_Victoria_secundaria$Año_curso, "2" , "SEGUNDO")
Notas_Victoria_secundaria$Año_curso <- str_replace(Notas_Victoria_secundaria$Año_curso, "3" , "TERCERO")
Notas_Victoria_secundaria$Año_curso <- str_replace(Notas_Victoria_secundaria$Año_curso, "4" , "CUARTO")
Notas_Victoria_secundaria$Año_curso <- str_replace(Notas_Victoria_secundaria$Año_curso, "5" , "QUINTO")
Notas_Victoria_secundaria$Año_curso <- str_replace(Notas_Victoria_secundaria$Año_curso, "6" , "SEXTO")
Notas_Victoria_secundaria$Año_curso <- str_replace(Notas_Victoria_secundaria$Año_curso, "7" , "SEPTIMO")

In [70]:
Notas_Victoria_1Año <- distinct(filter(Notas_Victoria_secundaria, Año_curso == "PRIMERO"))
math <- Notas_Victoria_1Año %>% group_by(CodigoUnicoEscolar,asignatura) %>%  filter(asignatura=="MATEMATICA")
math
tabla1 <- Notas_Victoria_1Año %>% group_by(asignatura) %>%  summarise(conteo=n())
tabla1

CodigoUnicoEscolar,departamento,localidad,EsPrivada,turno,curso,periodoEvaluatorio,Documento,asignatura,idalumno,...,IdModEnseñanza,Modalidad,ModEnseñanza,año,NotasPrimerT,NotasSegundoT,NotasTercerT,Promedio,Condición,Año_curso
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<int>,...,<int>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
9008373,VICTORIA,VICTORIA,N,NOCHE,1 - B 4000A,Tercer Trimestre,38258434,MATEMATICA,1118474,...,5,Secundaria,Comun,2023,9,4,4,5.67,Rinde,PRIMERO
9008373,VICTORIA,VICTORIA,N,NOCHE,1 - B 4000A,Tercer Trimestre,41610537,MATEMATICA,921414,...,5,Secundaria,Comun,2023,3,4,4,3.67,Rinde,PRIMERO
9008373,VICTORIA,VICTORIA,N,NOCHE,1 - B 4000A,Tercer Trimestre,42462646,MATEMATICA,907313,...,5,Secundaria,Comun,2023,3,4,4,3.67,Rinde,PRIMERO
9008373,VICTORIA,VICTORIA,N,NOCHE,1 - C 4000A,Tercer Trimestre,43147506,MATEMATICA,52402,...,5,Secundaria,Comun,2023,7,7,7,7.00,Aprobado,PRIMERO
9008373,VICTORIA,VICTORIA,N,NOCHE,1 - E 4000T,Tercer Trimestre,43755081,MATEMATICA,1158375,...,5,Secundaria,Comun,2023,6,5,4,5.00,Rinde,PRIMERO
9008373,VICTORIA,VICTORIA,N,NOCHE,1 - B 4000A,Tercer Trimestre,44281981,MATEMATICA,920575,...,5,Secundaria,Comun,2023,3,4,4,3.67,Rinde,PRIMERO
9008373,VICTORIA,VICTORIA,N,NOCHE,1 - B 4000A,Tercer Trimestre,44642545,MATEMATICA,1003828,...,5,Secundaria,Comun,2023,3,4,4,3.67,Rinde,PRIMERO
9008373,VICTORIA,VICTORIA,N,NOCHE,1 - C 4000A,Tercer Trimestre,44701349,MATEMATICA,1005929,...,5,Secundaria,Comun,2023,3,2,3,2.67,Rinde,PRIMERO
9008373,VICTORIA,VICTORIA,N,NOCHE,1 - C 4000A,Tercer Trimestre,44734257,MATEMATICA,67170,...,5,Secundaria,Comun,2023,5,5,5,5.00,Rinde,PRIMERO


asignatura,conteo
<chr>,<int>
ARTES VISUALES,524
BIOLOGIA,561
DIBUJO TECNICO,104
ECONOMIA,2
EDUCACION FISICA,497
ESPACIO DE LA ORIENTACION EN CIENCIAS NATURALES,1
ETICA,562
FISICA,32
FISICO Y QUIMICA,521


In [71]:
Notas_Victoria_2Año <- distinct(filter(Notas_Victoria_secundaria, Año_curso == "SEGUNDO"))
math <- Notas_Victoria_2Año %>% group_by(CodigoUnicoEscolar,asignatura) %>%  filter(asignatura=="MATEMATICA")
math
tabla1 <- Notas_Victoria_2Año %>% group_by(asignatura) %>%  summarise(conteo=n())
tabla1

CodigoUnicoEscolar,departamento,localidad,EsPrivada,turno,curso,periodoEvaluatorio,Documento,asignatura,idalumno,...,IdModEnseñanza,Modalidad,ModEnseñanza,año,NotasPrimerT,NotasSegundoT,NotasTercerT,Promedio,Condición,Año_curso
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<int>,...,<int>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
9008373,VICTORIA,VICTORIA,N,NOCHE,2 - B 4000A,Tercer Trimestre,42972181,MATEMATICA,919478,...,5,Secundaria,Comun,2023,4,4,4,4.00,Rinde,SEGUNDO
9003552,VICTORIA,VICTORIA,N,NOCHE,2 - 2 4000E,Tercer Trimestre,43630792,MATEMATICA,68106,...,5,Secundaria,Comun,2023,4,6,8,6.00,Aprobado,SEGUNDO
9008373,VICTORIA,VICTORIA,N,NOCHE,2 - B 4000A,Tercer Trimestre,43630863,MATEMATICA,1021255,...,7,Secundaria,Técnica,2023,6,5,4,5.00,Rinde,SEGUNDO
9008373,VICTORIA,VICTORIA,N,NOCHE,2 - A 4000E,Tercer Trimestre,44282032,MATEMATICA,142195,...,5,Secundaria,Comun,2023,8,8,8,8.00,Aprobado,SEGUNDO
9008373,VICTORIA,VICTORIA,N,NOCHE,2 - D 4000T,Tercer Trimestre,44841680,MATEMATICA,66566,...,5,Secundaria,Comun,2023,3,3,4,3.33,Rinde,SEGUNDO
9003552,VICTORIA,VICTORIA,N,NOCHE,2 - 2 4000E,Tercer Trimestre,44841722,MATEMATICA,553544,...,5,Secundaria,Comun,2023,5,8,8,7.00,Aprobado,SEGUNDO
9003552,VICTORIA,VICTORIA,N,NOCHE,2 - 2 4000E,Tercer Trimestre,44921342,MATEMATICA,64501,...,5,Secundaria,Comun,2023,6,6,6,6.00,Aprobado,SEGUNDO
9003552,VICTORIA,VICTORIA,N,NOCHE,2 - 2 4000E,Tercer Trimestre,45335214,MATEMATICA,574138,...,5,Secundaria,Comun,2023,4,4,4,4.00,Rinde,SEGUNDO
9003561,VICTORIA,VICTORIA,N,MAÑANA,2 - 2 CBC,Tercer Trimestre,45387105,MATEMATICA,149406,...,5,Secundaria,Comun,2023,8,10,7,8.33,Aprobado,SEGUNDO


asignatura,conteo
<chr>,<int>
ARTES VISUALES,567
BIDIMENSION I Y II,12
BIOLOGIA,591
DIBUJO TECNICO,123
ECONOMIA,8
EDUCACION FISICA,546
ETICA,600
FISICO Y QUIMICA,461
FISICO Y QUIMICA),124


In [72]:
Notas_Victoria_3Año <- distinct(filter(Notas_Victoria_secundaria, Año_curso == "TERCERO"))
math <- Notas_Victoria_3Año %>% group_by(CodigoUnicoEscolar,asignatura) %>%  filter(asignatura=="MATEMATICA")
math
tabla1 <- Notas_Victoria_3Año %>% group_by(asignatura) %>%  summarise(conteo=n())
tabla1

CodigoUnicoEscolar,departamento,localidad,EsPrivada,turno,curso,periodoEvaluatorio,Documento,asignatura,idalumno,...,IdModEnseñanza,Modalidad,ModEnseñanza,año,NotasPrimerT,NotasSegundoT,NotasTercerT,Promedio,Condición,Año_curso
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<int>,...,<int>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
9008373,VICTORIA,VICTORIA,N,NOCHE,3 - B 4000A,Tercer Trimestre,38052214,MATEMATICA,907312,...,5,Secundaria,Comun,2023,8,9,9,8.67,Aprobado,TERCERO
9008373,VICTORIA,VICTORIA,N,NOCHE,3 - B 4000A,Tercer Trimestre,39717060,MATEMATICA,907314,...,5,Secundaria,Comun,2023,9,10,10,9.67,Aprobado,TERCERO
9003552,VICTORIA,VICTORIA,N,NOCHE,3 - 2 4000E,Tercer Trimestre,43935016,MATEMATICA,1122646,...,5,Secundaria,Comun,2023,4,6,8,6.00,Aprobado,TERCERO
9003552,VICTORIA,VICTORIA,N,NOCHE,3 - 2 4000E,Tercer Trimestre,44282070,MATEMATICA,46286,...,5,Secundaria,Comun,2023,6,6,8,6.67,Aprobado,TERCERO
9003552,VICTORIA,VICTORIA,N,NOCHE,3 - 2 4000E,Tercer Trimestre,44282107,MATEMATICA,46049,...,7,Secundaria,Técnica,2023,8,7,9,8.00,Aprobado,TERCERO
9009816,VICTORIA,VICTORIA,N,ROTATIVO,3 - A Cs. Soc y Hum 485/10C,Tercer Trimestre,44495859,MATEMATICA,904507,...,7,Secundaria,Técnica,2023,6,6,6,6.00,Aprobado,TERCERO
9003552,VICTORIA,VICTORIA,N,NOCHE,3 - 2 4000E,Tercer Trimestre,44554903,MATEMATICA,156640,...,5,Secundaria,Comun,2023,8,8,8,8.00,Aprobado,TERCERO
9003552,VICTORIA,VICTORIA,N,NOCHE,3 - 1 4000E,Tercer Trimestre,44554938,MATEMATICA,255767,...,7,Secundaria,Técnica,2023,6,4,4,4.67,Rinde,TERCERO
9008373,VICTORIA,VICTORIA,N,NOCHE,3 - C 4000A,Tercer Trimestre,44841664,MATEMATICA,172192,...,5,Secundaria,Comun,2023,6,6,8,6.67,Aprobado,TERCERO


asignatura,conteo
<chr>,<int>
ARTES VISUALES,472
BIDIMENSION I Y II,11
BIOLOGIA,448
DERECHO,17
DIBUJO TECNICO,131
ECONOMIA,22
EDUCACION FISICA,462
ETICA,476
FISICO Y QUIMICA,369


In [73]:
Notas_Victoria_4Año <- distinct(filter(Notas_Victoria_secundaria, Año_curso == "CUARTO"))
math <- Notas_Victoria_4Año %>% group_by(CodigoUnicoEscolar,asignatura) %>%  filter(asignatura=="MATEMATICA")
math
tabla1 <- Notas_Victoria_4Año %>% group_by(asignatura) %>%  summarise(conteo=n())
tabla1

CodigoUnicoEscolar,departamento,localidad,EsPrivada,turno,curso,periodoEvaluatorio,Documento,asignatura,idalumno,...,IdModEnseñanza,Modalidad,ModEnseñanza,año,NotasPrimerT,NotasSegundoT,NotasTercerT,Promedio,Condición,Año_curso
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<int>,...,<int>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
9008049,VICTORIA,VICTORIA,N,TARDE,4 - A 401Ef,Tercer Trimestre,44734253,MATEMATICA,348473,...,5,Secundaria,Comun,2023,10,8,8,8.67,Aprobado,CUARTO
9007815,VICTORIA,VICTORIA,N,MAÑANA,4 - A 3322N,Tercer Trimestre,45335181,MATEMATICA,260914,...,5,Secundaria,Comun,2023,7,8,8,7.67,Aprobado,CUARTO
9007674,VICTORIA,VICTORIA,N,TARDE,4 - A 3322T,Tercer Trimestre,45387176,MATEMATICA,207065,...,5,Secundaria,Comun,2023,5,4,4,4.33,Rinde,CUARTO
9007722,VICTORIA,VICTORIA,N,MAÑANA,4 - A 3322A,Tercer Trimestre,45615887,MATEMATICA,402262,...,5,Secundaria,Comun,2023,4,4,4,4.00,Rinde,CUARTO
9003561,VICTORIA,VICTORIA,N,MAÑANA,4 - 1 3322T,Tercer Trimestre,45688560,MATEMATICA,1124044,...,5,Secundaria,Comun,2023,5,7,7,6.33,Aprobado,CUARTO
9004029,VICTORIA,VICTORIA,N,TARDE,4 - 1 TQ TQUIMIC,Tercer Trimestre,46390124,MATEMATICA,188448,...,7,Secundaria,Técnica,2023,3,4,1,2.67,Rinde,CUARTO
9007944,VICTORIA,VICTORIA,N,TARDE,4 - 1 3322S,Tercer Trimestre,46390134,MATEMATICA,256347,...,5,Secundaria,Comun,2023,6,6,3,5.00,Rinde,CUARTO
9003561,VICTORIA,VICTORIA,N,MAÑANA,4 - 1 3322S,Tercer Trimestre,46390166,MATEMATICA,192849,...,5,Secundaria,Comun,2023,6,6,6,6.00,Aprobado,CUARTO
9007677,VICTORIA,VICTORIA,N,MAÑANA,4 - A 3322N,Tercer Trimestre,46465018,MATEMATICA,34182,...,5,Secundaria,Comun,2023,9,10,10,9.67,Aprobado,CUARTO


asignatura,conteo
<chr>,<int>
ARTES VISUALES,379
BIOLOGIA,451
CATEQUESIS,26
CIENCIAS AGROPECUARIAS,41
CULTURA Y COMUNICACION,9
DIBUJO TECNICO,65
ECONOMIA,365
EDUCACION FISICA,459
ETICA,463


In [74]:
Notas_Victoria_5Año <- distinct(filter(Notas_Victoria_secundaria, Año_curso == "QUINTO"))
math <- Notas_Victoria_5Año %>% group_by(CodigoUnicoEscolar,asignatura) %>%  filter(asignatura=="MATEMATICA")
math
tabla1 <- Notas_Victoria_5Año %>% group_by(asignatura) %>%  summarise(conteo=n())
tabla1

CodigoUnicoEscolar,departamento,localidad,EsPrivada,turno,curso,periodoEvaluatorio,Documento,asignatura,idalumno,...,IdModEnseñanza,Modalidad,ModEnseñanza,año,NotasPrimerT,NotasSegundoT,NotasTercerT,Promedio,Condición,Año_curso
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<int>,...,<int>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
9007815,VICTORIA,VICTORIA,N,MAÑANA,5 - A 3322N,Tercer Trimestre,43755044,MATEMATICA,67542,...,5,Secundaria,Comun,2023,5,5,5,5.00,Rinde,QUINTO
9005244,VICTORIA,VICTORIA,N,TARDE,5 - A 3322E,Tercer Trimestre,44495849,MATEMATICA,130559,...,5,Secundaria,Comun,2023,6,5,7,6.00,Aprobado,QUINTO
9003552,VICTORIA,VICTORIA,N,TARDE,5 - 4 3322A,Tercer Trimestre,44701319,MATEMATICA,255880,...,5,Secundaria,Comun,2023,4,5,4,4.33,Rinde,QUINTO
9003561,VICTORIA,VICTORIA,N,MAÑANA,5 - 1 3322N,Tercer Trimestre,44841708,MATEMATICA,183753,...,5,Secundaria,Comun,2023,4,4,1,3.00,Rinde,QUINTO
9007722,VICTORIA,VICTORIA,N,MAÑANA,5 - A 3322A,Tercer Trimestre,45615904,MATEMATICA,614968,...,5,Secundaria,Comun,2023,8,8,8,8.00,Aprobado,QUINTO
9007944,VICTORIA,VICTORIA,N,TARDE,5 - 1 3322S,Tercer Trimestre,45615909,MATEMATICA,140763,...,5,Secundaria,Comun,2023,4,6,8,6.00,Aprobado,QUINTO
9008043,VICTORIA,VICTORIA,N,TARDE,5 - A 3322E,Tercer Trimestre,45754009,MATEMATICA,209018,...,5,Secundaria,Comun,2023,7,7,7,7.00,Aprobado,QUINTO
9003552,VICTORIA,VICTORIA,N,TARDE,5 - 3 3322S,Tercer Trimestre,45754138,MATEMATICA,796220,...,5,Secundaria,Comun,2023,8,8,8,8.00,Aprobado,QUINTO
9007722,VICTORIA,VICTORIA,N,MAÑANA,5 - A 3322A,Tercer Trimestre,45754171,MATEMATICA,614984,...,5,Secundaria,Comun,2023,6,7,6,6.33,Aprobado,QUINTO


asignatura,conteo
<chr>,<int>
BIOLOGIA,341
C.A.D,16
CATEQUESIS,48
CIENCIAS AGROPECUARIAS,18
COMUNICACION,9
DERECHO,46
DIBUJO TECNICO,16
ECONOMIA,137
ECONOMIA REGIONAL Y ADMIN AGROPECUARIA POL,9


In [75]:
Notas_Victoria_6Año <- distinct(filter(Notas_Victoria_secundaria, Año_curso == "SEXTO"))
math <- Notas_Victoria_6Año %>% group_by(CodigoUnicoEscolar,asignatura) %>%  filter(asignatura=="MATEMATICA")
math
tabla1 <- Notas_Victoria_6Año %>% group_by(asignatura) %>%  summarise(conteo=n())
tabla1

CodigoUnicoEscolar,departamento,localidad,EsPrivada,turno,curso,periodoEvaluatorio,Documento,asignatura,idalumno,...,IdModEnseñanza,Modalidad,ModEnseñanza,año,NotasPrimerT,NotasSegundoT,NotasTercerT,Promedio,Condición,Año_curso
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<int>,...,<int>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
9007674,VICTORIA,VICTORIA,N,TARDE,6 - A 3322T,Tercer Trimestre,43147474,MATEMATICA,150938,...,5,Secundaria,Comun,2023,6,6,7,6.33,Aprobado,SEXTO
9008046,VICTORIA,VICTORIA,N,TARDE,6 - A 3322S,Tercer Trimestre,44282012,MATEMATICA,143263,...,5,Secundaria,Comun,2023,8,9,8,8.33,Aprobado,SEXTO
9007815,VICTORIA,VICTORIA,N,MAÑANA,6 - A 3322N,Tercer Trimestre,44554932,MATEMATICA,156678,...,5,Secundaria,Comun,2023,6,8,8,7.33,Aprobado,SEXTO
9003561,VICTORIA,VICTORIA,N,MAÑANA,6 - 2 3322N,Tercer Trimestre,44841598,MATEMATICA,76172,...,5,Secundaria,Comun,2023,7,9,8,8.00,Aprobado,SEXTO
9003561,VICTORIA,VICTORIA,N,MAÑANA,6 - 1 3322T,Tercer Trimestre,44916714,MATEMATICA,567776,...,5,Secundaria,Comun,2023,5,4,3,4.00,Rinde,SEXTO
9007815,VICTORIA,VICTORIA,N,MAÑANA,6 - A 3322N,Tercer Trimestre,44921279,MATEMATICA,160096,...,5,Secundaria,Comun,2023,6,6,6,6.00,Aprobado,SEXTO
9007674,VICTORIA,VICTORIA,N,TARDE,6 - A 3322T,Tercer Trimestre,44978892,MATEMATICA,361443,...,5,Secundaria,Comun,2023,6,6,6,6.00,Aprobado,SEXTO
9007674,VICTORIA,VICTORIA,N,TARDE,6 - A 3322T,Tercer Trimestre,45046691,MATEMATICA,664400,...,5,Secundaria,Comun,2023,6,6,7,6.33,Aprobado,SEXTO
9003561,VICTORIA,VICTORIA,N,MAÑANA,6 - 1 3322N,Tercer Trimestre,45046763,MATEMATICA,121107,...,5,Secundaria,Comun,2023,5,6,7,6.00,Aprobado,SEXTO


asignatura,conteo
<chr>,<int>
ARTES VISUALES,4
BIOLOGIA,157
C.A.D,24
CATEQUESIS,46
CIENCIAS AGROPECUARIAS,10
COMUNICACION,6
"COMUNICACION, EDUCACION Y NUEVAS TECNOLOGIAS",6
DERECHO,83
DIBUJO DE MAQUINAS,24


In [76]:
Notas_Victoria_7Año <- distinct(filter(Notas_Victoria_secundaria, Año_curso == "SEPTIMO"))
math <- Notas_Victoria_7Año %>% group_by(CodigoUnicoEscolar,asignatura) %>%  filter(asignatura=="MATEMATICA")
math
tabla1 <- Notas_Victoria_7Año %>% group_by(asignatura) %>%  summarise(conteo=n())
tabla1

CodigoUnicoEscolar,departamento,localidad,EsPrivada,turno,curso,periodoEvaluatorio,Documento,asignatura,idalumno,...,IdModEnseñanza,Modalidad,ModEnseñanza,año,NotasPrimerT,NotasSegundoT,NotasTercerT,Promedio,Condición,Año_curso
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<int>,...,<int>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
9004029,VICTORIA,VICTORIA,N,MAÑANA,7 - 1 TPA TEPABYS,Tercer Trimestre,45387220,MATEMATICA,59425,...,8,Secundaria,Agrotécnica,2023,7,6,6,6.33,Aprobado,SEPTIMO
9004029,VICTORIA,VICTORIA,N,MAÑANA,7 - 1 TPA TEPABYS,Tercer Trimestre,45753990,MATEMATICA,29392,...,8,Secundaria,Agrotécnica,2023,7,6,7,6.67,Aprobado,SEPTIMO
9004029,VICTORIA,VICTORIA,N,MAÑANA,7 - 1 TPA TEPABYS,Tercer Trimestre,45945216,MATEMATICA,313245,...,8,Secundaria,Agrotécnica,2023,5,6,7,6.00,Aprobado,SEPTIMO
9004029,VICTORIA,VICTORIA,N,MAÑANA,7 - 1 TPA TEPABYS,Tercer Trimestre,46079894,MATEMATICA,187387,...,8,Secundaria,Agrotécnica,2023,8,8,8,8.00,Aprobado,SEPTIMO
9004029,VICTORIA,VICTORIA,N,MAÑANA,7 - 1 TPA TEPABYS,Tercer Trimestre,46079948,MATEMATICA,126239,...,8,Secundaria,Agrotécnica,2023,10,10,10,10.00,Aprobado,SEPTIMO
9004029,VICTORIA,VICTORIA,N,MAÑANA,7 - 1 TPA TEPABYS,Tercer Trimestre,46079960,MATEMATICA,1013155,...,8,Secundaria,Agrotécnica,2023,5,4,7,5.33,Rinde,SEPTIMO
9004029,VICTORIA,VICTORIA,N,MAÑANA,7 - 1 TPA TEPABYS,Tercer Trimestre,46079966,MATEMATICA,82496,...,8,Secundaria,Agrotécnica,2023,8,9,9,8.67,Aprobado,SEPTIMO
9004029,VICTORIA,VICTORIA,N,MAÑANA,7 - 1 TPA TEPABYS,Tercer Trimestre,46390095,MATEMATICA,570521,...,8,Secundaria,Agrotécnica,2023,7,8,8,7.67,Aprobado,SEPTIMO
9004029,VICTORIA,VICTORIA,N,MAÑANA,7 - 1 TPA TEPABYS,Tercer Trimestre,46465067,MATEMATICA,173097,...,8,Secundaria,Agrotécnica,2023,8,5,8,7.00,Aprobado,SEPTIMO


asignatura,conteo
<chr>,<int>
CIENCIAS AGROPECUARIAS,57
ECONOMIA,15
EDUCACION FISICA,39
FISICA,15
FORMULACION DE PROYECTOS PRODUCTIVOS,15
LABORATORIO DE ENSAYOS DE MAQUINAS Y MOTORES,17
MANTENIMIENTO Y REPARACION DE EQUIPOS,17
MAQUINAS TERMICAS,17
MAQUINAS Y ELEMENTOS DE TRANSPORTE,17


In [77]:
dim(Notas_Victoria_secundaria)

[1] 51591    27

In [ ]:
write_xlsx(Notas_Victoria_secundaria, "C:/Users/ayrto/OneDrive/Escritorio/Exploración de Datos Multivariados/Trabajo Practico/VICTORIA_BASE_SECUNDARIA_LIMPIA.xlsx")